In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
import gensim
from gensim import corpora, models, matutils
import re
from nltk.corpus import stopwords
from sklearn.metrics import precision_recall_fscore_support
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

In [4]:
dataset=pd.read_csv("Data/ITcomments.csv")
dataset.head()

,comment,Media Inputs,Navigation,APPs,Sound Quality,Audio System,Bluetooth,Voice Recognition,Audio Controls,Connectivity,...,AWD,Sun Visor,Tinted Windows,ABS,Sunroof,Amenities,IT,User Friendliness,ITS,EODO
0,I WOULD LIKE TO HAVE A STANDARD PLUG AVAILABLE...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,TO ACTUALLY HAVE A NAVIGATION SYSTEM BUILT IN,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,THE NISSAN CONNECT APPS DO NOT WORK WELL AND T...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,THE VEHICLE DOES NOT HAVE A NAVIGATION SYSTEM ...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,BEING ABLE TO TAKE THE BASS OUT OF THE DOOR SP...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [113]:
documents=dataset["comment"].values
stoplist = set(stopwords.words("english"))
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]
#remove words used once
all_tokens = sum(texts, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
texts = [[word for word in text if word not in tokens_once] for text in texts]

In [114]:
#Build a dictionary - a frequency distribution of integer IDs representing words
dictionary = corpora.Dictionary(texts)
#Build a vector space corpus - use the dictionary to translate
# word vectors into sparse feature vectors
corpus = [dictionary.doc2bow(text) for text in texts]
# Train a model that will "reward" tokens that are distinctive to documents - comment out to test non-tfIdf
tfidf = models.TfidfModel(corpus)
# creates a lazy evaluating wrapper around corpus - doesn't transform the whole corpus
#  at once because we don't want to load the whole corpus into memory.
corpus_tfidf = tfidf[corpus]

In [115]:
#create LSI model
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=400)
corpus_lsi = lsi[corpus]

In [116]:
col=dataset.columns.values
catList=[]
data=dataset.values
for i in range(24192):
    l=[]
    for j in range(len(col)):
        if data[i,j]==1:
            l.append(col[j])
    catList.append(l)
print(catList[0:2])
m=MultiLabelBinarizer()
y=m.fit_transform(catList)
print(y[:2])
ftwo_scorer = make_scorer(fbeta_score, beta=2)

IndexError: index 24191 is out of bounds for axis 0 with size 24191

In [ ]:
y=dataset["EODO"].values
Xtrain, Xtest, ytrain, ytest=train_test_split(corpus_lsi,y,test_size=.66)
sparseLsi=matutils.corpus2csc(Xtrain).transpose()
#classifier = OneVsRestClassifier(SVC(kernel='linear',decision_function_shape=None)) #Acc~75%
#classifier = OneVsRestClassifier(RandomForestClassifier()) #Acc ~ 45%
classifier = OneVsRestClassifier(LogisticRegression()) #Acc ~74%

classifier.fit(sparseLsi,ytrain)

In [ ]:
sparseLsiTest=matutils.corpus2csc(Xtest).transpose()
print(classifier.score(sparseLsiTest,ytest))

In [ ]:
classifiers=[]
transformedCorpus=matutils.corpus2csc(corpus_lsi).transpose()
for c in col[1:]:
    print(c)
    y=dataset[c].values
    Xtrain, Xtest, ytrain, ytest=train_test_split(transformedCorpus,y,test_size=.5,stratify=y)
    classifier=SVC(kernel='linear',C=13.75)
    classifier.fit(Xtrain,ytrain)
    print("{} Accuracy: {}".format(c,classifier.score(Xtest,ytest)))
    print(precision_recall_fscore_support(ytest, classifier.predict(Xtest)))
    classifiers.append(classifier)

In [75]:
y=dataset['Air Bags'].values
transformedCorpus=matutils.corpus2csc(corpus_lsi).transpose()
Xtrain, Xtest, ytrain, ytest=train_test_split(transformedCorpus,y,stratify=y)
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [97]:
grid = GridSearchCV(SVC(kernel='linear'), param_grid={'C': [13,13.25,13.5,13.75,14]}, scoring=ftwo_scorer)
grid.fit(Xtrain,ytrain)
grid.score(Xtest,ytest)
print(grid.best_params_)
print(grid.best_score_)
#precision_recall_fscore_support(ytest, classifier.predict(Xtest))

{'C': 13.5}
0.416584164514


In [73]:
print(grid.best_params_)

{'C': 10}
